# Project M2 - Cell Type Classification with Morphology features

## Getting started with the Allen's data set

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("MorphFeatures.csv",index_col=0)
print(df.shape)
df.head(5) #show some lines on the data from the first record.

(670, 31)


,average_bifurcation_angle_local,average_bifurcation_angle_remote,average_contraction,average_diameter,average_fragmentation,average_parent_daughter_ratio,hausdorff_dimension,id,max_branch_order,max_euclidean_distance,...,scale_factor_y,scale_factor_z,soma_surface,specimen_id,superseded,tags,total_length,total_surface,total_volume,dendrite_type
0,82.727781,NaN,0.864267,0.345092,20.723077,0.964510,NaN,491119743,6.0,99.779724,...,0.1144,0.28,435.740270,478107198,False,3D Neuron Reconstruction morphology,1666.082926,1803.875644,167.343086,aspiny
1,82.506680,NaN,0.903890,0.634047,105.277778,0.862183,NaN,546781359,3.0,432.383110,...,0.1144,0.28,1446.587725,502367941,False,3D Neuron Reconstruction morphology,2277.259374,4543.139073,921.571895,spiny
2,77.536678,NaN,0.863104,0.417929,73.666667,0.926633,NaN,537042261,6.0,373.630444,...,0.1144,0.28,287.118123,515771244,False,3D Neuron Reconstruction morphology,3589.339062,4704.910407,582.285423,spiny
3,76.583222,NaN,0.900537,0.400396,95.979167,0.942049,NaN,689123605,11.0,943.382549,...,0.1144,0.28,180.994813,561435279,False,3D Neuron Reconstruction morphology,5416.228778,6814.933290,740.722806,spiny
4,72.019250,NaN,0.873518,0.227626,47.535714,1.000000,NaN,657879305,5.0,186.218009,...,0.1144,0.28,55.055236,591268268,False,3D Neuron Reconstruction morphology,1659.465869,1185.773462,69.144146,aspiny


The cell type is determined by the dendrite type in the last column of the data set. Ignore the samples of minority type called "sparsely spiny".

In [2]:
df.dropna(axis=1,inplace=True) # Drop columns with Nan values
df = df.drop_duplicates(subset=['specimen_id']) # drop duplicated of specimen_id

# Get rid of sparsely spiny cells
df = df[df.dendrite_type!='sparsely spiny'] #keep all the data that 'dendrite_type' is not 'sparsely spiny'
print(df.shape)
print(df.columns)
df.head(5)

(619, 29)
Index(['average_bifurcation_angle_local', 'average_contraction',
       'average_diameter', 'average_fragmentation',
       'average_parent_daughter_ratio', 'id', 'max_branch_order',
       'max_euclidean_distance', 'max_path_distance',
       'neuron_reconstruction_type', 'number_bifurcations', 'number_branches',
       'number_nodes', 'number_stems', 'number_tips', 'overall_depth',
       'overall_height', 'overall_width', 'scale_factor_x', 'scale_factor_y',
       'scale_factor_z', 'soma_surface', 'specimen_id', 'superseded', 'tags',
       'total_length', 'total_surface', 'total_volume', 'dendrite_type'],
      dtype='object')


,average_bifurcation_angle_local,average_contraction,average_diameter,average_fragmentation,average_parent_daughter_ratio,id,max_branch_order,max_euclidean_distance,max_path_distance,neuron_reconstruction_type,...,scale_factor_y,scale_factor_z,soma_surface,specimen_id,superseded,tags,total_length,total_surface,total_volume,dendrite_type
0,82.727781,0.864267,0.345092,20.723077,0.964510,491119743,6.0,99.779724,126.593790,dendrite-only,...,0.1144,0.28,435.740270,478107198,False,3D Neuron Reconstruction morphology,1666.082926,1803.875644,167.343086,aspiny
1,82.506680,0.903890,0.634047,105.277778,0.862183,546781359,3.0,432.383110,496.831994,dendrite-only,...,0.1144,0.28,1446.587725,502367941,False,3D Neuron Reconstruction morphology,2277.259374,4543.139073,921.571895,spiny
2,77.536678,0.863104,0.417929,73.666667,0.926633,537042261,6.0,373.630444,436.958952,dendrite-only,...,0.1144,0.28,287.118123,515771244,False,3D Neuron Reconstruction morphology,3589.339062,4704.910407,582.285423,spiny
3,76.583222,0.900537,0.400396,95.979167,0.942049,689123605,11.0,943.382549,989.448317,full,...,0.1144,0.28,180.994813,561435279,False,3D Neuron Reconstruction morphology,5416.228778,6814.933290,740.722806,spiny
4,72.019250,0.873518,0.227626,47.535714,1.000000,657879305,5.0,186.218009,221.639502,full,...,0.1144,0.28,55.055236,591268268,False,3D Neuron Reconstruction morphology,1659.465869,1185.773462,69.144146,aspiny


## Feature Engineering

In [3]:
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

df_full = df.drop(columns=['id','neuron_reconstruction_type','scale_factor_x', 'scale_factor_y','scale_factor_z','specimen_id','superseded','tags'])
print(df_full.shape)
df_full.head()

(619, 21)


,average_bifurcation_angle_local,average_contraction,average_diameter,average_fragmentation,average_parent_daughter_ratio,max_branch_order,max_euclidean_distance,max_path_distance,number_bifurcations,number_branches,...,number_stems,number_tips,overall_depth,overall_height,overall_width,soma_surface,total_length,total_surface,total_volume,dendrite_type
0,82.727781,0.864267,0.345092,20.723077,0.964510,6.0,99.779724,126.593790,33,73,...,7,40,51.4886,140.506829,136.267522,435.740270,1666.082926,1803.875644,167.343086,aspiny
1,82.506680,0.903890,0.634047,105.277778,0.862183,3.0,432.383110,496.831994,9,23,...,5,14,92.6671,566.701220,370.170045,1446.587725,2277.259374,4543.139073,921.571895,spiny
2,77.536678,0.863104,0.417929,73.666667,0.926633,6.0,373.630444,436.958952,21,46,...,4,25,65.4696,425.897625,381.015114,287.118123,3589.339062,4704.910407,582.285423,spiny
3,76.583222,0.900537,0.400396,95.979167,0.942049,11.0,943.382549,989.448317,24,52,...,4,28,99.9139,1217.694976,524.550156,180.994813,5416.228778,6814.933290,740.722806,spiny
4,72.019250,0.873518,0.227626,47.535714,1.000000,5.0,186.218009,221.639502,14,32,...,4,18,54.3718,172.075941,261.459057,55.055236,1659.465869,1185.773462,69.144146,aspiny


In [4]:
X = abs(df_full.iloc[:,:-1]) # Need to take absolute value for SelectKBest to work
y = df_full.iloc[:,-1]       # iloc Purely integer-location based indexing for selection by position.

In [5]:
bestfeatures = SelectKBest(score_func=chi2)
fit = bestfeatures.fit(X,y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(df_full.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns

print(featureScores.nlargest(20,'Score'))  #print 20 best features

                              Specs          Score
18                    total_surface  539939.690284
17                     total_length  264251.289979
10                     number_nodes  220597.718099
19                     total_volume  115805.409932
7                 max_path_distance   30418.233712
6            max_euclidean_distance   27774.577274
14                   overall_height   23066.856685
9                   number_branches    2410.349432
15                    overall_width    2214.133645
13                    overall_depth    1994.393610
12                      number_tips    1213.162125
8               number_bifurcations    1201.211153
16                     soma_surface     905.144986
5                  max_branch_order     556.265637
11                     number_stems      64.412472
3             average_fragmentation      55.236331
0   average_bifurcation_angle_local      16.120053
2                  average_diameter       0.834203
4     average_parent_daughter_r

In [6]:
pick_feats = list(featureScores.nlargest(20,'Score').Specs) # make a list of the ten best features
pick_feats.append('dendrite_type') # add dendrite_type to the list

df_small = df[pick_feats] # Make a new DataFrame with our selected features
df_small.head(5)

,total_surface,total_length,number_nodes,total_volume,max_path_distance,max_euclidean_distance,overall_height,number_branches,overall_width,overall_depth,...,number_bifurcations,soma_surface,max_branch_order,number_stems,average_fragmentation,average_bifurcation_angle_local,average_diameter,average_parent_daughter_ratio,average_contraction,dendrite_type
0,1803.875644,1666.082926,1470,167.343086,126.593790,99.779724,140.506829,73,136.267522,51.4886,...,33,435.740270,6.0,7,20.723077,82.727781,0.345092,0.964510,0.864267,aspiny
1,4543.139073,2277.259374,2011,921.571895,496.831994,432.383110,566.701220,23,370.170045,92.6671,...,9,1446.587725,3.0,5,105.277778,82.506680,0.634047,0.862183,0.903890,spiny
2,4704.910407,3589.339062,3137,582.285423,436.958952,373.630444,425.897625,46,381.015114,65.4696,...,21,287.118123,6.0,4,73.666667,77.536678,0.417929,0.926633,0.863104,spiny
3,6814.933290,5416.228778,4652,740.722806,989.448317,943.382549,1217.694976,52,524.550156,99.9139,...,24,180.994813,11.0,4,95.979167,76.583222,0.400396,0.942049,0.900537,spiny
4,1185.773462,1659.465869,1406,69.144146,221.639502,186.218009,172.075941,32,261.459057,54.3718,...,14,55.055236,5.0,4,47.535714,72.019250,0.227626,1.000000,0.873518,aspiny


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, scale
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

Using TensorFlow backend.


Split the dataset into two groups for training and testing.

In [8]:
train = df_small.sample(frac=0.8,random_state=111) # random pick 80% for training
test = df_small.drop(train.index)
print(train.shape)
print(test.shape)

(495, 21)
(124, 21)


Train and test in different groups.

In [9]:
X = train.values[:,:-1] # put all columns from first to last but not including the last one in X
Y = train.values[:,-1]  # put last columns in y
x = test.values[:,:-1]  # put all columns from first to last but not including the last one in X
y = test.values[:,-1]   # put last columns in y

lb = LabelBinarizer()  # Binarize labels in a one-vs-all fashion
Y_b = lb.fit_transform(Y) # convert values in y into binary labels
y_b = lb.fit_transform(y)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
training_y = encoder.transform(Y)
testing_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_training = np_utils.to_categorical(training_y)
dummy_testing = np_utils.to_categorical(testing_y)

log_reg = LogisticRegression(penalty="l2")
log_reg.fit(X,Y_b)

y_pred = log_reg.predict(x)  #Predict class labels for samples in X.
print("Model accuracy:", accuracy_score(y_b,y_pred))  # Calculate the accuracy comparing the predicted labels with the ground truth

Model accuracy: 0.8951612903225806


/opt/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Neural Network for Morphology

In [10]:
def baseline_model():
    model = Sequential()
    model.add(Dense(22, input_dim=20, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model 
X = scale(X)
model = baseline_model();
model.fit(X,dummy_training,epochs=100,verbose=1)
y_pred = model.predict_classes(x)
print("Model accuracy:", accuracy_score(y_b,y_pred)) 

/opt/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


Epoch 1/100
495/495 [==============================] - 0s 539us/step - loss: 0.6501 - acc: 0.5960
Epoch 2/100
495/495 [==============================] - 0s 62us/step - loss: 0.5206 - acc: 0.7333
Epoch 3/100
495/495 [==============================] - 0s 60us/step - loss: 0.4482 - acc: 0.8020
Epoch 4/100
495/495 [==============================] - 0s 102us/step - loss: 0.4031 - acc: 0.8283
Epoch 5/100
495/495 [==============================] - 0s 75us/step - loss: 0.3696 - acc: 0.8485
Epoch 6/100
495/495 [==============================] - 0s 73us/step - loss: 0.3464 - acc: 0.8606
Epoch 7/100
495/495 [==============================] - 0s 105us/step - loss: 0.3275 - acc: 0.8707
Epoch 8/100
495/495 [==============================] - 0s 101us/step - loss: 0.3129 - acc: 0.8727
Epoch 9/100
495/495 [==============================] - 0s 74us/step - loss: 0.2996 - acc: 0.8808
Epoch 10/100
495/495 [==============================] - 0s 67us/step - loss: 0.2897 - acc: 0.8788
Epoch 11/100
495/495 [===

495/495 [==============================] - 0s 43us/step - loss: 0.1144 - acc: 0.9636
Epoch 85/100
495/495 [==============================] - 0s 42us/step - loss: 0.1130 - acc: 0.9636
Epoch 86/100
495/495 [==============================] - 0s 40us/step - loss: 0.1138 - acc: 0.9636
Epoch 87/100
495/495 [==============================] - 0s 41us/step - loss: 0.1114 - acc: 0.9657
Epoch 88/100
495/495 [==============================] - 0s 40us/step - loss: 0.1109 - acc: 0.9657
Epoch 89/100
495/495 [==============================] - 0s 40us/step - loss: 0.1099 - acc: 0.9657
Epoch 90/100
495/495 [==============================] - 0s 41us/step - loss: 0.1091 - acc: 0.9657
Epoch 91/100
495/495 [==============================] - 0s 41us/step - loss: 0.1078 - acc: 0.9657
Epoch 92/100
495/495 [==============================] - 0s 41us/step - loss: 0.1075 - acc: 0.9657
Epoch 93/100
495/495 [==============================] - 0s 40us/step - loss: 0.1061 - acc: 0.9657
Epoch 94/100
495/495 [===========

In [11]:
df_ef = pd.read_csv("ElecPhyFeatures.csv",index_col=0)       # Read electrophysiology data
print(df.shape)
print(df_ef.shape)

df_cb = pd.merge(df,df_ef,how='inner',left_on='specimen_id', right_on='specimen_id',suffixes=('_mp','')) # Combine two data frames
print(df_cb.shape)

df_cb = df_cb.drop(columns='dendrite_type_mp') # dendrite_type was duplicated while merging two dataframes, drop out one of them.
print(df_cb.shape)
print(df_cb.columns)
df_cb.head(5)

(619, 29)
(2333, 57)
(619, 85)
(619, 84)
Index(['average_bifurcation_angle_local', 'average_contraction',
       'average_diameter', 'average_fragmentation',
       'average_parent_daughter_ratio', 'id_mp', 'max_branch_order',
       'max_euclidean_distance', 'max_path_distance',
       'neuron_reconstruction_type', 'number_bifurcations', 'number_branches',
       'number_nodes', 'number_stems', 'number_tips', 'overall_depth',
       'overall_height', 'overall_width', 'scale_factor_x', 'scale_factor_y',
       'scale_factor_z', 'soma_surface', 'specimen_id', 'superseded', 'tags',
       'total_length', 'total_surface', 'total_volume', 'adaptation',
       'avg_isi', 'electrode_0_pa', 'f_i_curve_slope',
       'fast_trough_t_long_square', 'fast_trough_t_ramp',
       'fast_trough_t_short_square', 'fast_trough_v_long_square',
       'fast_trough_v_ramp', 'fast_trough_v_short_square', 'has_burst',
       'has_delay', 'has_pause', 'id', 'input_resistance_mohm', 'latency',
       'peak_t_lo

,average_bifurcation_angle_local,average_contraction,average_diameter,average_fragmentation,average_parent_daughter_ratio,id_mp,max_branch_order,max_euclidean_distance,max_path_distance,neuron_reconstruction_type,...,trough_t_short_square,trough_v_long_square,trough_v_ramp,trough_v_short_square,upstroke_downstroke_ratio_long_square,upstroke_downstroke_ratio_ramp,upstroke_downstroke_ratio_short_square,vm_for_sag,vrest,dendrite_type
0,82.727781,0.864267,0.345092,20.723077,0.964510,491119743,6.0,99.779724,126.593790,dendrite-only,...,1.047640,-59.812500,-59.437504,-66.718750,2.625872,2.969025,1.909216,-77.718758,-63.952812,aspiny
1,82.506680,0.903890,0.634047,105.277778,0.862183,546781359,3.0,432.383110,496.831994,dendrite-only,...,1.114696,-58.125004,-57.864586,-66.443753,4.322536,4.272167,3.659369,-98.937508,-66.457214,spiny
2,77.536678,0.863104,0.417929,73.666667,0.926633,537042261,6.0,373.630444,436.958952,dendrite-only,...,1.236256,-56.000000,-57.979170,-69.356253,5.076478,5.069481,3.992479,-85.312500,-68.378876,spiny
3,76.583222,0.900537,0.400396,95.979167,0.942049,689123605,11.0,943.382549,989.448317,full,...,1.096650,-55.031250,-55.989587,-63.726564,3.445772,3.264711,3.118708,-83.625000,-62.545727,spiny
4,72.019250,0.873518,0.227626,47.535714,1.000000,657879305,5.0,186.218009,221.639502,full,...,1.377193,-60.656254,-54.562500,-72.192708,1.255817,1.051984,1.371654,-85.093750,-71.524811,aspiny


## Combined Features Logistic

In [12]:
df_cb = df_cb.drop(columns='neuron_reconstruction_type')
df_cb = df_cb.drop(columns='tags')
df_cb = df_cb.drop(columns='superseded')
df_cb.dropna(inplace=True)
combined_X = df_cb.iloc[:,:-1] # Need to take absolute value for SelectKBest to work
combined_y = df_cb.iloc[:,-1]       # iloc Purely integer-location based indexing for selection by position.

combined_train = df_cb.sample(frac=0.8,random_state=111) # random pick 80% for training
combined_test = df_cb.drop(combined_train.index)

combined_X = combined_train.values[:,:-1] # put all columns from first to last but not including the last one in X
combined_Y = combined_train.values[:,-1]  # put last columns in y
combined_x = combined_test.values[:,:-1]  # put all columns from first to last but not including the last one in X
combined_y = combined_test.values[:,-1]   # put last columns in y

combined_lb = LabelBinarizer()  # Binarize labels in a one-vs-all fashion
combined_Y_b = combined_lb.fit_transform(combined_Y) # convert values in y into binary labels
combined_y_b = combined_lb.fit_transform(combined_y)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(combined_y)
combined_training_y = encoder.transform(combined_Y)
combined_testing_y = encoder.transform(combined_y)
# convert integers to dummy variables (i.e. one hot encoded)
combined_dummy_training = np_utils.to_categorical(combined_training_y)
combined_dummy_testing = np_utils.to_categorical(combined_testing_y)

combined_log_reg = LogisticRegression(penalty="l2")
combined_log_reg.fit(combined_X,combined_Y_b)

combined_y_pred = combined_log_reg.predict(combined_x)  #Predict class labels for samples in X.

print("Model accuracy:", accuracy_score(combined_y_b,combined_y_pred)) 

Model accuracy: 0.7368421052631579


/opt/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Combined Features Neural Network, For Accuracy measurement I use the built in Keras validation_split which splits the dataset according to the fraction you implement. In this case I use .2. It gets me an accuracy of 96.77%

In [13]:
warnings.filterwarnings('ignore')
def baseline_model():
    model = Sequential()
    model.add(Dense(83, input_dim=80, activation='relu'))
    model.add(Dense(83, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model 
combined_X = scale(combined_X)
model = baseline_model();
model.fit(combined_X,combined_dummy_training, validation_split = .2, epochs=200,verbose=1)
combined_y_pred = model.predict(combined_x)

Train on 122 samples, validate on 31 samples
Epoch 1/200
122/122 [==============================] - 0s 2ms/step - loss: 0.6523 - acc: 0.6066 - val_loss: 0.5017 - val_acc: 0.7742
Epoch 2/200
122/122 [==============================] - 0s 82us/step - loss: 0.3674 - acc: 0.8443 - val_loss: 0.3383 - val_acc: 0.9355
Epoch 3/200
122/122 [==============================] - 0s 80us/step - loss: 0.2434 - acc: 0.9098 - val_loss: 0.2690 - val_acc: 0.9355
Epoch 4/200
122/122 [==============================] - 0s 73us/step - loss: 0.1750 - acc: 0.9426 - val_loss: 0.2262 - val_acc: 0.9355
Epoch 5/200
122/122 [==============================] - 0s 79us/step - loss: 0.1235 - acc: 0.9836 - val_loss: 0.1956 - val_acc: 0.9355
Epoch 6/200
122/122 [==============================] - 0s 77us/step - loss: 0.0930 - acc: 0.9918 - val_loss: 0.1723 - val_acc: 0.9355
Epoch 7/200
122/122 [==============================] - 0s 74us/step - loss: 0.0716 - acc: 0.9918 - val_loss: 0.1547 - val_acc: 0.9355
Epoch 8/200
122/12

122/122 [==============================] - 0s 85us/step - loss: 8.5210e-04 - acc: 1.0000 - val_loss: 0.0557 - val_acc: 0.9677
Epoch 62/200
122/122 [==============================] - 0s 91us/step - loss: 8.2559e-04 - acc: 1.0000 - val_loss: 0.0555 - val_acc: 0.9677
Epoch 63/200
122/122 [==============================] - 0s 72us/step - loss: 8.0232e-04 - acc: 1.0000 - val_loss: 0.0551 - val_acc: 0.9677
Epoch 64/200
122/122 [==============================] - 0s 71us/step - loss: 7.7566e-04 - acc: 1.0000 - val_loss: 0.0548 - val_acc: 0.9677
Epoch 65/200
122/122 [==============================] - 0s 76us/step - loss: 7.5407e-04 - acc: 1.0000 - val_loss: 0.0545 - val_acc: 0.9677
Epoch 66/200
122/122 [==============================] - 0s 77us/step - loss: 7.3225e-04 - acc: 1.0000 - val_loss: 0.0543 - val_acc: 0.9677
Epoch 67/200
122/122 [==============================] - 0s 72us/step - loss: 7.1018e-04 - acc: 1.0000 - val_loss: 0.0542 - val_acc: 0.9677
Epoch 68/200
122/122 [==================

Epoch 120/200
122/122 [==============================] - 0s 71us/step - loss: 2.2795e-04 - acc: 1.0000 - val_loss: 0.0458 - val_acc: 0.9677
Epoch 121/200
122/122 [==============================] - 0s 68us/step - loss: 2.2419e-04 - acc: 1.0000 - val_loss: 0.0457 - val_acc: 0.9677
Epoch 122/200
122/122 [==============================] - 0s 154us/step - loss: 2.2065e-04 - acc: 1.0000 - val_loss: 0.0456 - val_acc: 0.9677
Epoch 123/200
122/122 [==============================] - 0s 69us/step - loss: 2.1728e-04 - acc: 1.0000 - val_loss: 0.0454 - val_acc: 0.9677
Epoch 124/200
122/122 [==============================] - 0s 74us/step - loss: 2.1363e-04 - acc: 1.0000 - val_loss: 0.0454 - val_acc: 0.9677
Epoch 125/200
122/122 [==============================] - 0s 73us/step - loss: 2.1016e-04 - acc: 1.0000 - val_loss: 0.0452 - val_acc: 0.9677
Epoch 126/200
122/122 [==============================] - 0s 72us/step - loss: 2.0718e-04 - acc: 1.0000 - val_loss: 0.0451 - val_acc: 0.9677
Epoch 127/200
122/1

122/122 [==============================] - 0s 76us/step - loss: 1.0458e-04 - acc: 1.0000 - val_loss: 0.0415 - val_acc: 0.9677
Epoch 179/200
122/122 [==============================] - 0s 78us/step - loss: 1.0339e-04 - acc: 1.0000 - val_loss: 0.0414 - val_acc: 0.9677
Epoch 180/200
122/122 [==============================] - 0s 78us/step - loss: 1.0228e-04 - acc: 1.0000 - val_loss: 0.0414 - val_acc: 0.9677
Epoch 181/200
122/122 [==============================] - 0s 73us/step - loss: 1.0116e-04 - acc: 1.0000 - val_loss: 0.0413 - val_acc: 0.9677
Epoch 182/200
122/122 [==============================] - 0s 72us/step - loss: 1.0008e-04 - acc: 1.0000 - val_loss: 0.0413 - val_acc: 0.9677
Epoch 183/200
122/122 [==============================] - 0s 73us/step - loss: 9.8898e-05 - acc: 1.0000 - val_loss: 0.0412 - val_acc: 0.9677
Epoch 184/200
122/122 [==============================] - 0s 77us/step - loss: 9.7899e-05 - acc: 1.0000 - val_loss: 0.0411 - val_acc: 0.9677
Epoch 185/200
122/122 [===========

# Additional Methods (Ensemble, etc) 

In [14]:
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier, AdaBoostClassifier, VotingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC

In [15]:
X = train.values[:,:-1] # put all columns from first to last but not including the last one in X
Y = train.values[:,-1]  # put last columns in y
x = test.values[:,:-1]  # put all columns from first to last but not including the last one in X
y = test.values[:,-1]   # put last columns in y

lb = LabelBinarizer()  # Binarize labels in a one-vs-all fashion
Y_b = lb.fit_transform(Y) # convert values in y into binary labels
y_b = lb.fit_transform(y)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
training_y = encoder.transform(Y)
testing_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_training = np_utils.to_categorical(training_y)
dummy_testing = np_utils.to_categorical(testing_y)

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X,Y_b)

y_pred = rf.predict(x)  #Predict class labels for samples in X.

print("Model accuracy:", accuracy_score(y_b,y_pred)) 

Model accuracy: 0.9112903225806451


In [16]:
et = ExtraTreesClassifier()
et.fit(X,Y_b)

y_pred = et.predict(x)  #Predict class labels for samples in X.

print("Model accuracy:", accuracy_score(y_b,y_pred))

Model accuracy: 0.9112903225806451


In [37]:
ada = AdaBoostClassifier(n_estimators = 27)
ada.fit(X, Y_b)

y_pred = ada.predict(x)

print("Model accuracy:", accuracy_score(y_b,y_pred))

Model accuracy: 0.9354838709677419


In [18]:
knn = KNeighborsClassifier()
knn.fit(X,Y_b)

y_pred = knn.predict(x)  #Predict class labels for samples in X.

print("Model accuracy:", accuracy_score(y_b,y_pred))

Model accuracy: 0.8145161290322581


In [19]:
svc = SVC()
svc.fit(X,Y_b)

y_pred = svc.predict(x)  #Predict class labels for samples in X.

print("Model accuracy:", accuracy_score(y_b,y_pred))

Model accuracy: 0.5564516129032258


In [20]:
rg = RidgeClassifier()
rg.fit(X,Y_b)

y_pred = rg.predict(x)  #Predict class labels for samples in X.

print("Model accuracy:", accuracy_score(y_b,y_pred))

Model accuracy: 0.9112903225806451


# Extra Credit - Voting 

In [21]:
warnings.filterwarnings('ignore')
from sklearn.model_selection import KFold
seed = 12345
kfold = KFold(n_splits=5, shuffle=True, random_state=seed)
rf = RandomForestClassifier(n_estimators=10)
rf.fit(X,Y_b)
y_pred1 = rf.predict(x)  #Predict class labels for samples in X.
print("Model accuracy:", accuracy_score(y_b,y_pred1)) 

et = ExtraTreesClassifier()
et.fit(X,Y_b)
y_pred2 = et.predict(x)  #Predict class labels for samples in X.
print("Model accuracy:", accuracy_score(y_b,y_pred2))

knn = KNeighborsClassifier()
knn.fit(X,Y_b)
y_pred3 = knn.predict(x)  #Predict class labels for samples in X.
print("Model accuracy:", accuracy_score(y_b,y_pred3))

clf_array = [rf, et, knn]

for clf in clf_array:
    vanilla_scores = cross_val_score(clf, X, Y_b, cv=kfold, n_jobs=-1)
    bagging_clf = BaggingClassifier(clf, max_samples=0.25, max_features=1.0, random_state=seed)
    # max_samples: The proportion of samples to draw from X to train each base estimator.
    # max_features: The proportion of features to draw from X to train each base estimator.
    bagging_scores = cross_val_score(bagging_clf, X, Y_b, cv=10, n_jobs=-1)
    
    print("Mean of: {1:.3f}, std: (+/-) {2:.3f} [{0}]".format(clf.__class__.__name__, vanilla_scores.mean()*100, vanilla_scores.std()*100))
    print("Mean of: {1:.3f}, std: (+/-) {2:.3f} [Bagging {0}]\n".format(clf.__class__.__name__, bagging_scores.mean()*100, bagging_scores.std()*100))
    
# Set up voting
eclf = VotingClassifier(estimators=[('Random Forests',rf), ('Extra Trees',et), ('KNeighbors',knn)],
                        voting='hard')

for clf, label in zip([rf, et, knn], ['Random Forest','Extra Trees','KNeighbors']):
    scores = cross_val_score(clf, X, Y_b, cv=10, scoring='accuracy')
    print("Mean: {0:.3f}, std: (+/-) {1:.3f} [{2}]".format(scores.mean()*100, scores.std()*100, label))

Model accuracy: 0.9112903225806451
Model accuracy: 0.8951612903225806
Model accuracy: 0.8145161290322581
Mean of: 90.101, std: (+/-) 2.157 [RandomForestClassifier]
Mean of: 87.664, std: (+/-) 3.084 [Bagging RandomForestClassifier]

Mean of: 88.283, std: (+/-) 3.037 [ExtraTreesClassifier]
Mean of: 88.685, std: (+/-) 2.067 [Bagging ExtraTreesClassifier]

Mean of: 79.596, std: (+/-) 1.616 [KNeighborsClassifier]
Mean of: 79.019, std: (+/-) 4.384 [Bagging KNeighborsClassifier]

Mean: 88.693, std: (+/-) 3.133 [Random Forest]
Mean: 90.926, std: (+/-) 4.194 [Extra Trees]
Mean: 80.203, std: (+/-) 4.910 [KNeighbors]


In [22]:
warnings.filterwarnings('ignore')
# Set up ensemble voting for bagging
ebclf_array = []

for clf in clf_array:
    ebclf_array.append(BaggingClassifier(clf, max_samples=0.25, max_features=1.0, random_state=seed))

v_eclf = VotingClassifier(estimators=list(zip(['Bagging Random Forest','Bagging Extra Trees','Bagging KNeighbors'],ebclf_array)),  voting='hard')
ebclf_array.append(v_eclf)

for clf, label in zip(ebclf_array, ['Bagging Random Forest', 'Bagging Extra Trees', 'Bagging KNeighbors']):
    scores = cross_val_score(clf, X, Y_b, cv=kfold, scoring='accuracy')
    print("Mean: {0:.3f}, std: (+/-) {1:.3f} [{2}]".format(scores.mean(), scores.std(), label))

Mean: 0.875, std: (+/-) 0.015 [Bagging Random Forest]
Mean: 0.873, std: (+/-) 0.031 [Bagging Extra Trees]
Mean: 0.776, std: (+/-) 0.022 [Bagging KNeighbors]
